# NFL Data Scraper

In [1]:
import requests
import re
import pandas as pd
import datetime as dt
from bs4 import BeautifulSoup
from dateutil.parser import parse

In [2]:
teams = {'atl':'Atlanta Falcons','buf':'Buffalo Bills','car':'Carolina Panthers','chi':'Chicago Bears','cin':'Cincinnati Bengals','cle':'Cleveland Browns','clt':'Indianapolis Colts','crd':'Arizona Cardinals','dal':'Dallas Cowboys','den':'Denver Broncos','det':'Detroit Lions','gnb':'Green Bay Packers','htx':'Houston Texans','jax':'Jacksonville Jaguars','kan':'Kansas City Chiefs','mia':'Miami Dolphins','min':'Minnesota Vikings','nor':'New Orleans Saints','nwe':'New England Patriots','nyg':'New York Giants','nyj':'New York Jets','oti':'Tennessee Titans','phi':'Philadelphia Eagles','pit':'Pittsburgh Steelers','rai':['Oakland Raiders','Las Vegas Raiders'],'ram':['St. Louis Rams','Los Angeles Rams'],'rav':'Baltimore Ravens','sdg':['San Diego Chargers','Los Angeles Chargers'],'sea':'Seattle Seahawks','sfo':'San Francisco 49ers','tam':'Tampa Bay Buccaneers','was':'Washington Redskins'}
years = [str(i) for i in range(2010,2020)]

In [3]:
yearcols = ['Date','Home/Away','Opponent','Points','Points_Opp','Yds_Off_Pass','Yds_Off_Rush','Yds_Def_Pass','Yds_Def_Rush','TD','TD_on_Def','FG_Made','FG_Att','RZ_Conv','RZ_Att','RZ_Def_Conv','RZ_Def_Att','Possession','Plays','TO_Gained','TO_Lost','Yds_Pen','Sacks_Def','Tackles_Loss','Yds_per_Kickret','Yds_per_Puntret']
dfyear=pd.DataFrame(index=teams.keys(),columns=yearcols)

In [4]:
comm = re.compile('<!--|-->')

#for year in years:
#    for team in teams.keys():
year = years[0]
team = next(iter(teams.keys()))
url = 'https://www.pro-football-reference.com/teams/'+team+'/'+year+'/gamelog/'
res = requests.get(url)
#print(res.status_code)
#soup = BeautifulSoup(comm.sub("",res.text),'html.parser')

In [5]:
tableID = 'gamelog'+year
table=pd.read_html(comm.sub("",res.text),attrs={'id':tableID},flavor='bs4')
table=table[0]

table['Game'] = range(1,17)
table.set_index('Game',inplace=True)

L0 = list(table.columns.get_level_values(0))
L0_mod = ['Event' if 'Unnamed' in L0[i] else L0[i] for i in range(len(L0))]
L0_mod[-1] = 'Time'
L0_renmd = dict(zip(L0,L0_mod))
table.rename(columns=L0_renmd,level=0,inplace=True)

In [6]:
ind = pd.MultiIndex.from_product([list(teams.keys()),[i for i in range(1,17)]],names=['Team','Game'])
dfyear = pd.DataFrame(index=ind,columns=yearcols)

In [7]:
dfteam = dfyear.loc[team,:]

In [8]:
dconv = lambda x: parse(x + ' ' + str(int(year)+1)).strftime('%Y%m%d') if 'January' in x else parse(x + ' ' + year).strftime('%Y%m%d')
dfteam['Date'] = table['Event']['Date'].apply(dconv)

In [9]:
dfteam['Home/Away'][table.iloc[:,6] == '@'] = 'A'
dfteam['Home/Away'][table.iloc[:,6] != '@'] = 'H'

In [ ]:
dates=dfteam['Date']
for game in dfteam.index:
    if dfteam['Home/Away'][game] == 'H':
        gamecode = date+'0'+team
    elif dfteam['Home/Away'][game] == 'A':
        gamecode 

In [11]:
dfteam.head()

,Date,Home/Away,Opponent,Points,Points_Opp,Yds_Off_Pass,Yds_Off_Rush,Yds_Def_Pass,Yds_Def_Rush,TD,...,RZ_Def_Att,Possession,Plays,TO_Gained,TO_Lost,Yds_Pen,Sacks_Def,Tackles_Loss,Yds_per_Kickret,Yds_per_Puntret
Game,,,,,,,,,,,,,,,,,,,,,
1,09122010,A,Pittsburgh Steelers,9,15,237,58,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,09192010,H,Arizona Cardinals,41,7,223,221,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,09262010,A,New Orleans Saints,27,24,215,202,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10032010,H,San Francisco 49ers,16,14,259,98,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10102010,A,Cleveland Browns,20,10,173,165,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
table.head()

Event                                                                \
      Week  Day          Date Unnamed: 3_level_1 Unnamed: 4_level_1   OT   
Game                                                                       
1        1  Sun  September 12           boxscore                  L   OT   
2        2  Sun  September 19           boxscore                  W  NaN   
3        3  Sun  September 26           boxscore                  W   OT   
4        4  Sun     October 3           boxscore                  W  NaN   
5        5  Sun    October 10           boxscore                  W  NaN   

                                             Score      ... Scoring          \
     Unnamed: 6_level_1                  Opp    Tm Opp  ...     FGA XPM XPA   
Game                                                    ...                   
1                     @  Pittsburgh Steelers     9  15  ...       4   0   0   
2                   NaN    Arizona Cardinals    41   7  ...       2   5   5   
3                     @   New Orleans Saints    27  24  ...       2   3   3   
4                   NaN  San Francisco 49ers    16  14  ...       3   1   1   
5                     @     Cleveland Browns    20  10  ...       3   2   2   

     Punting       Downs                      Time  
         Pnt  Yds 3DConv 3DAtt 4DConv 4DAtt    ToP  
Game                                                
1          7  281      6    16      0     0  30:29  
2          2   68     11    17      1     2  39:13  
3          4  158      9    18      2     3  45:50  
4          5  165      5    14      1     1  29:19  
5          4  173      6    15      0     0  30:06  

[5 rows x 36 columns]